In [1]:
import numpy as np

time_bob = np.array([ #[time, empirica; number of observations]
  [10 , 4],
  [20 , 6],
  [30 , 10],
  [40 , 20],
  [50 , 40],
  [60 , 11],
  [70 , 5],
  [80 , 4]
]) 

time_alice = np.array([ #[time, empirica; number of observations]
  [10 , 4],
  [20 , 5],
  [30 , 6],
  [40 , 7],
  [50 , 10],
  [60 , 8],
  [70 , 6],
  [80 , 2]       
])

print(time_bob)
print(time_bob.shape)

[[10  4]
 [20  6]
 [30 10]
 [40 20]
 [50 40]
 [60 11]
 [70  5]
 [80  4]]
(8, 2)


In [2]:
import numpy.random as rd
rd.seed(0)

rd.choice(time_alice[:,0],100,p=time_alice[:,1]/np.sum(time_alice[:,1]) )

array([50, 60, 50, 50, 40, 50, 40, 70, 80, 40, 60, 50, 50, 70, 10, 20, 10,
       60, 60, 70, 80, 60, 50, 60, 20, 50, 20, 70, 50, 40, 30, 60, 40, 50,
       10, 50, 50, 50, 70, 60, 40, 40, 60, 10, 60, 60, 30, 20, 40, 40, 50,
       40, 80, 20, 30, 20, 50, 30, 50, 30, 20, 20, 50, 20, 30, 40, 60, 20,
       70, 20, 80, 50, 80, 50, 60, 10, 30, 20, 30, 20, 40, 40, 10, 60, 50,
       30, 50, 20, 50, 70, 40, 60, 20, 60, 30, 20, 50, 10, 60, 10])

In [6]:
def next_event(*args):
    vals = [v for v in args if v is not None]
    if len(vals)==0:
        return None
    return min(vals)

assert next_event(1,2,3) == 1
assert next_event(4,2,None) == 2
assert next_event(None,None) == None

In [7]:
class ProductionLine:
    alice_action_end = 0  # time measured in seconds
    bob_action_end = None   
    alice_has_box = False #Alice holds or prepares box for Bob
    bob_has_box = False   #Bob works with box

    #storage = 0 # we can model boxes going to storage

    tick  = 0       # simulation step
    clock = 0       # time in simulated world in seconda
    completed_boxes = 0 # completed boxes

    def __init__(self,time_bob,time_alice): #konstruktor
        self.time_alice = time_alice
        self.time_bob = time_bob        
        self.p_time_alice = time_alice[:,1]/np.sum(time_alice[:,1])
        self.p_time_bob = time_bob[:,1]/np.sum(time_bob[:,1]) 
    
    def step(self):
        #Selecting the next event to happen
        # Calculating the time of the next event to happen
        # Check if this event triggers another conditiona; action such as leaving the queue
        
        #Alice & Bob
        #Events: Alice completed job, Bob completed job
        #Conditional events: 
        # - Alice starts job (needs to have free hands)
        # - Bob starts job (needs to have free hands and a box must be available)
        
        self.clock = min_vals(self.alice_action_end,self.bob_action_end)
        
        if self.alice_has_box and self.bob_action_end is not None and self.bob_action_end > self.alice_action_end:
            print(self.clock, "Alice needs to wait for Bob ")
            self.clock = self.bob_action_end 
        
        if not self.alice_has_box :
            self.alice_has_box = True
            self.alice_action_end = self.clock + \
                    rd.choice(self.time_alice[:,0],p=self.p_time_alice)
            print(self.clock, "Alice starts doing new box for the time ", self.alice_action_end)


        if self.bob_has_box and self.bob_action_end <= self.clock:
            print(self.clock, "Bob finishes the box")
            self.completed_boxes += 1
            self.bob_action_end = None
            self.bob_has_box = False

            
        if self.alice_has_box and (self.alice_action_end is None or self.alice_action_end <= self.clock):
            print(self.clock, "Alice is ready to pass box to Bob ")
            if not self.bob_has_box:            
                self.bob_action_end = self.clock + \
                   rd.choice(self.time_bob[:,0],p=self.p_time_bob)
                print(self.clock, "Bob takes the box from Alice and will be ready at ",self.bob_action_end )
                self.bob_has_box = True
                self.alice_has_box = False
          
        self.tick = self.tick + 1 #increase step size
        
            
    def status(self):
        print("### tick=",self.tick,"clock=",self.clock,"Alice has box", self.alice_has_box,"Alice end=",
              self.alice_action_end,
              "Bob end=",self.bob_action_end, "Bob has box=",self.bob_has_box,
              "completed_boxes=",self.completed_boxes )
        #"storage=",self.storage 
        

In [8]:
rd.seed(10)
# Starting the model
m = ProductionLine(time_bob, time_alice)  # Creating the object with ___init___ constructor
for step in range(40):  
    m.step()
    m.status()

NameError: name 'min_vals' is not defined